# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

#### Answer

As per the paper linked, the researchers performed 3 iterations of CRISP-DM 

1. Iteration 1 --> 79354 contacts with 6499 successful conversions to deposit (8% success rate)
2. Iteration 2 --> 55817 contacts with 6499 successful deposits (11.6% success rate)
3. Iteration 3 --> 45211 contacts with 5289 successful deposits (11.7% success rate)

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd

In [2]:
df_bank = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df_bank.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
# Helper function to get all details about the dataframe
def display_column_info(df):
  """
  Given a dataframe df, returns the statistics of this dataframe
  """

  # check if input parameter is actually a df
  if not isinstance(df, pd.DataFrame):
    prtint("Input parameter is not a dataframe")
    return None

  column_names = []
  data_types = []
  nan_count_list = []
  nan_percentage_list = []
  unique_count_list = []
  data_range_list = []

  for column in df.columns:
    column_names.append(column)
    data_types.append(df[column].dtype)
    nan_count = ((df[column] == 0).sum()) if pd.api.types.is_numeric_dtype(df[column]) else df[column].isnull().sum()
    nan_count_list.append(nan_count)
    nan_percentage_list = (nan_count / len(df)) * 100
    unique_count_list.append(df[column].nunique())

    if pd.api.types.is_numeric_dtype(df[column]):
      min_val = df[column].min()
      max_val = df[column].max()
      data_range_list.append(f"{min_val} - {max_val}")
    else:
      data_range_list.append(df[column].unique())

  column_info_df = pd.DataFrame(
      {'column_name': column_names,
       'data_type': data_types,
       'nan_or_zero_count': nan_count_list,
       'nan_or_zero_percentage': nan_percentage_list,
       'unique_count': unique_count_list,
       'data_range': data_range_list
      })

  column_info_df.set_index('column_name', inplace=True)
  rows, cols = df.shape
  print("Number of rows: ", rows)
  print("Number of columns: ", cols)
  display(column_info_df)
  return None
    

In [5]:
display_column_info(df_bank)

Number of rows:  41188
Number of columns:  21


,data_type,nan_or_zero_count,nan_or_zero_percentage,unique_count,data_range
column_name,,,,,
age,int64,0,0.0,78,17 - 98
job,object,0,0.0,12,"[housemaid, services, admin., blue-collar, tec..."
marital,object,0,0.0,4,"[married, single, divorced, unknown]"
education,object,0,0.0,8,"[basic.4y, high.school, basic.6y, basic.9y, pr..."
default,object,0,0.0,3,"[no, unknown, yes]"
housing,object,0,0.0,3,"[no, yes, unknown]"
loan,object,0,0.0,3,"[no, yes, unknown]"
contact,object,0,0.0,2,"[telephone, cellular]"
month,object,0,0.0,10,"[may, jun, jul, aug, oct, nov, dec, mar, apr, ..."


In [6]:
df_bank.query("duration == 0")

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
6251,39,admin.,married,high.school,no,yes,no,telephone,may,tue,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
23031,59,management,married,university.degree,no,yes,no,cellular,aug,tue,...,10,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,no
28063,53,blue-collar,divorced,high.school,no,yes,no,cellular,apr,fri,...,3,999,0,nonexistent,-1.8,93.075,-47.1,1.479,5099.1,no
33015,31,blue-collar,married,basic.9y,no,no,no,cellular,may,mon,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.299,5099.1,no


In [7]:
# since there are only 4 data samples with duration == 0
# and for all these samples the target variable 'y' is 'no'
# we can safely drop this samples

df_bank_filtered = df_bank.query("duration != 0")

display_column_info(df_bank_filtered)

Number of rows:  41184
Number of columns:  21


,data_type,nan_or_zero_count,nan_or_zero_percentage,unique_count,data_range
column_name,,,,,
age,int64,0,0.0,78,17 - 98
job,object,0,0.0,12,"[housemaid, services, admin., blue-collar, tec..."
marital,object,0,0.0,4,"[married, single, divorced, unknown]"
education,object,0,0.0,8,"[basic.4y, high.school, basic.6y, basic.9y, pr..."
default,object,0,0.0,3,"[no, unknown, yes]"
housing,object,0,0.0,3,"[no, yes, unknown]"
loan,object,0,0.0,3,"[no, yes, unknown]"
contact,object,0,0.0,2,"[telephone, cellular]"
month,object,0,0.0,10,"[may, jun, jul, aug, oct, nov, dec, mar, apr, ..."


In [8]:
df_bank_filtered['y'].value_counts()

y
no     36544
yes     4640
Name: count, dtype: int64

The provided bank data has
1. 41,188 rows
2. 21 columns with 20 features and 1 target variable 'y'
3. Target variable only has 2 unique values, 'yes' and 'no'
4. There are no NaN values
5. The column 'previous' has 35563 instances with value 0, which should not be considered as NaN / invalid value. It is highly possible that a lot of contacts were only done once.
6. The 'duration' column had 4 values with 0, it is almost 0.009% of data. Also the target variable 'y' for all of these is "no". so decided to drop these samples.
7. Now the new data is 41184 samples, with 4640 yes values, 11.2% success rate


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

#### Answer

The Business Objective is:
  - Given the data, build a prediction model to identify if a new customer will choose to opt for a deposit (success rate) if contacted. Also understand what can be changed/improved to enhance the success rate of customers opting for a long term deposit

In [10]:
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

### Problem 9: Score the Model

What is the accuracy of your model?

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions